# PROPUESTA ALGORITMO DE SELECCIÓN

Este ejercicio tiene como base un archivo de excel en el cual se establece la cantidad de programas ofertados (hoja Cupos) y la información de los aspirantes; tales como ID, Puntaje, y opciones.   
  
Para efectos prácticos se establecen 10 programas con un total de 55 cupos ofertados y 200 aspirantes.


## Resumen de las etapas de selección:


### Ordenar los Aspirantes por Puntaje:
Los aspirantes se ordenan por su puntaje de mayor a menor para priorizar a aquellos con los puntajes más altos en el proceso de selección.

### Inicializar Contadores de Cupos:
Se establecen contadores para cada programa para llevar un registro de los cupos disponibles.

### Proceso de Admisión:
- Se itera sobre la lista de aspirantes ordenada.
- Para cada aspirante, se intenta asignar un cupo en su primera opción. Si no hay cupos disponibles, se continúa con la siguiente opción hasta la quinta opción.
- Si se asigna un cupo, se decrementa el contador correspondiente al programa.

### Generar Listas de Espera:
- Si un aspirante no obtiene un cupo en ninguna de sus opciones, se le añade a la lista de espera de **cada** una de sus opciones.
- Esto resulta en que un aspirante puede aparecer en múltiples listas de espera.

### Creación de DataFrames de Resultados:
- Se generan dos conjuntos de DataFrames: uno para los admitidos y otro para las listas de espera.
- En los DataFrames de lista de espera, cada programa tiene su propia columna, y los aspirantes que no fueron admitidos se listan en todas las opciones que habían seleccionado y donde no fueron admitidos.
- Se guardan los resultados un en un archivos de Excel.



# Código

In [32]:
import pandas as pd


# Cargar los datos desde el archivo Excel; las hojas 'Cupos' y 'Aspirantes' en DataFrames
file_path = 'informacion.xlsx'
cupos_df = pd.read_excel(file_path, sheet_name='Cupos')
aspirantes_df = pd.read_excel(file_path, sheet_name='Aspirantes')

In [8]:
#Ordenar los aspirantes por puntaje de mayor a menor
aspirantes_df_sorted = aspirantes_df.sort_values(by='Puntaje', ascending=False)

In [10]:
#Inicializar un diccionario para llevar la cuenta de los cupos disponibles por programa
cupos_disponibles = cupos_df.set_index('PROGRAMAS')['CUPOS'].to_dict()

In [12]:
#Inicializar listas para los admitidos y las listas de espera
admitidos = []
listas_de_espera = {programa: [] for programa in cupos_disponibles}

In [15]:
# Función para intentar admitir a un aspirante en uno de sus programas de preferencia
def intentar_admision(aspirante, preferencias):
    for opcion in preferencias:
        if cupos_disponibles[opcion] > 0:
            # Admitir al aspirante en el programa de esta opción
            cupos_disponibles[opcion] -= 1
            admitidos.append((aspirante['ID'], opcion))
            return True
    # Si no fue admitido, añadir a las listas de espera
    for opcion in preferencias:
        listas_de_espera[opcion].append(aspirante['ID'])
    return False

In [16]:
# Procesar cada aspirante para intentar asignarle un cupo
for _, aspirante in aspirantes_df_sorted.iterrows():
    preferencias = [aspirante[f'Opción {i}'] for i in range(1, 6)] # 5 es el número total de opciones
    intentar_admision(aspirante, preferencias)

In [18]:
# Convertir los resultados en DataFrames para una mejor visualización
admitidos_df = pd.DataFrame(admitidos, columns=['ID', 'Programa Admitido'])
listas_de_espera_dfs = {programa: pd.DataFrame(ids, columns=['ID']) for programa, ids in listas_de_espera.items()}

In [33]:
# Crear un nuevo DataFrame para el Excel final con los programas como columnas
programas = list(cupos_disponibles.keys())
resultados_por_programa_df = pd.DataFrame(columns=programas)

# Añadir los admitidos al DataFrame final
for _, row in admitidos_df.iterrows():
    programa = row['Programa Admitido']
    aspirante_id = row['ID']
    # Añadir el ID del aspirante a la columna del programa correspondiente
    resultados_por_programa_df.at[resultados_por_programa_df[programa].size, programa] = aspirante_id

# Añadir los aspirantes en lista de espera al DataFrame final
for programa, df in listas_de_espera_dfs.items():
    for _, row in df.iterrows():
        aspirante_id = row['ID']
        # Añadir el ID del aspirante a la columna del programa correspondiente
        resultados_por_programa_df.at[resultados_por_programa_df[programa].size, programa] = aspirante_id

# Debido a que las columnas pueden tener diferentes longitudes, llenaremos los NaNs con una cadena vacía
resultados_por_programa_df = resultados_por_programa_df.fillna('')



In [44]:
# Función para limpiar y compactar las columnas, eliminando espacios en blanco o celdas no alfanuméricas
def clean_and_compact_columns(df):
    # Función para limpiar los espacios en blanco y otros caracteres no visibles
    def clean_cell(cell):
        if isinstance(cell, str):
            return cell.strip()
        return cell

    # Aplicar la limpieza a cada celda del DataFrame
    df_cleaned = df.applymap(clean_cell)

    # Función para desplazar hacia arriba los valores no vacíos de una columna
    def shift_up_non_empty(column):
        # Mantener solo las celdas que no son ni vacías ni contienen solo espacios en blanco
        non_empty = column[column.apply(lambda x: isinstance(x, str) and x.strip() != '')]
        # Restablecer el índice sin introducir NaNs al final de la serie
        return non_empty.reset_index(drop=True).reindex(range(df.shape[0]), fill_value='')

    # Aplicar el desplazamiento a cada columna del DataFrame limpiado
    df_compacted = df_cleaned.apply(shift_up_non_empty)

    return df_compacted

# Aplicar la limpieza y compactación al DataFrame
resultados_compacted_df = clean_and_compact_columns(resultados_por_programa_df)

C:\Users\57350\AppData\Local\Temp\ipykernel_6936\833194250.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df.applymap(clean_cell)


,Programa 1,Programa 2,Programa 3,Programa 4,Programa 5,Programa 6,Programa 7,Programa 8,Programa 9,Programa 10
0,Aspirante_157,Aspirante_65,Aspirante_80,Aspirante_37,Aspirante_6,Aspirante_13,Aspirante_61,Aspirante_153,Aspirante_161,Aspirante_124
1,Aspirante_179,Aspirante_174,Aspirante_147,Aspirante_170,Aspirante_146,Aspirante_120,Aspirante_41,Aspirante_30,Aspirante_112,Aspirante_60
2,Aspirante_132,Aspirante_198,Aspirante_115,Aspirante_89,Aspirante_139,Aspirante_107,Aspirante_171,Aspirante_172,Aspirante_74,Aspirante_152
3,Aspirante_56,Aspirante_169,Aspirante_198,Aspirante_49,Aspirante_9,Aspirante_100,Aspirante_167,Aspirante_151,Aspirante_81,Aspirante_141
4,Aspirante_21,Aspirante_131,Aspirante_102,Aspirante_179,Aspirante_148,Aspirante_84,Aspirante_83,Aspirante_129,Aspirante_33,Aspirante_64
...,...,...,...,...,...,...,...,...,...,...
775,,,,,,,,,,
776,,,,,,,,,,
777,,,,,,,,,,
778,,,,,,,,,,


In [47]:
# Guardar el DataFrame final en un archivo Excel
output_file_path = 'resultados_seleccion_universidad.xlsx'
resultados_compacted_df.to_excel(output_file_path, index=False)

"Listas de espera: Las y los aspirantes que tengan un puntaje de selección cercano al del último elegible de acuerdo con los cupos ofertados estarán en la lista de espera y serán contactados en el evento en el que se libere uno de los cupos. En el aplicativo de consulta de resultados se podrá verificar esta condición. Se les aconseja a las y los aspirantes con este estado que estén atentos al posible llamado de la IES correspondiente. El llamado se realizará conservando el orden del listado de aspirantes, dado por sus puntajes de selección."  
  
La descripción de la norma sugiere que un aspirante puede estar en lista de espera para un programa si su puntaje es cercano al del último admitido y podría estar en varias listas de espera si cumple esta condición en más de un programa. Es decir, según esta regla, es válido y esperado que un aspirante pueda aparecer en múltiples listas de espera.